In [ ]:
from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
from collections import OrderedDict
from IPython.display import display
from loguru import logger

from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config

# auto reload modules
%load_ext autoreload

# Paths definition
src_path = Path(f'{os.getenv("HOME")}/Nextcloud/Juanmi_MED_PSA/EURECAT/')
results_path: Path = src_path / 'results'
data_path: Path = src_path / 'data'

# filename_opt_result = '20240108_optimization_results.json'
# Debería ser un .csv al que se le hayan añadido las variables faltantes desde librescada:
# - J de variadores y medidor de potencia
# - FT-DES-002_VFD

filename_process_data = '20230807_solarMED.csv'

sample_rate = '60s'

# initial_datetime = '2024-01-08 10:55'
# final_datetime = '2024-01-08 14:00'

In [ ]:
# Load variables information
with open( Path("data") / 'variables_config.hjson') as f:
    vars_config = json.load(f)

# Load plot configuration
with open( Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)
    
# Read data from csv, the index column is the one named "time", which is not the first one
df = pd.read_csv(results_path / filename_process_data, parse_dates=True, index_col='time')

# Set UTC timezone
df = df.tz_localize('UTC')


In [ ]:
# Trim the dataframe to the desired time range

# df = df[:-19*60] # Time strings can be used, check past code
# Remove until 10:30

# df = df[initial_datetime:final_datetime]
# logger.info(f'Trimmed dataframe from {df.index[0]} to {df.index[-1]}')

In [ ]:
# Preprocessing

# Sample every `sample_rate` seconds to reduce the size of the dataframe
# df = df.resample(sample_rate).mean()

# Rename columns from signal_id to var_id
# When multiple var ids share the same signal_id, just have them duplicated



In [ ]:
# Calculate power consumptions from fitted curves

from calculations import power_consumption

df["Ce_dc"] = power_consumption(df["w_dc"].to_numpy(), actuator='fan_dc')
df["Ce_wct"] = power_consumption(df["w_wct"].to_numpy(), actuator='fan_wct')
df["Ce_c"] = power_consumption(df["q_c"].to_numpy(), actuator='pump_c')

In [ ]:
# Convert units to model units

In [ ]:
from phd_visualizations.test_timeseries import experimental_results_plot

# Load plot configuration here as well to not have to go back and forth
with open(Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, df_opt)